# Linear Probability Model

The purpose of this program is to regress a mortgage approval variable against race, ethnicity, gender, and other control variables found in HMDA data. Using the model below.

$P(Approval = 1 | Race/Ethnicity/Sex, \chi_j) = \beta_0 + \lambda_j * Race/Ethnic/Sex  + \beta_j * \chi_j + \mu $

Where $\lambda_j$ are the variables of interest, $/beta_j$ are the coefficients on the control variables, and $\chi_j$ are the control variables.

Variables of Interest
- White
- Black
- Asian
- Other
- Multi-Race Interactions
- Hispanic
- Non-Hispanic
- Hispanic and Race Interactions
- Male 
- Female

Control Variables
- Income (log)
- Loan to Value ratio
- Debt to Income ratio
- Loan Amount (log)
- Pre-Approval indicators

Fixed Effects - maybe include
- Lender
- Region Indicators by Community Tract  or county

Variables ommited in model to prevent perfect collinearity.
- White
- Non-Hispanic
- Male

Filters
- Loan Purpose
- Occupancy Type

Clustered Standard errors
- by Lender
- by Region
- by County

Other regressions to run that will use similar controls.
- Simplified Model(Just variables of interest)
- Restricted Model
- Interest Rates
- Denial Rates
- Fixed Effects Model
- Years other than 2019

In [1]:
import pandas as pd
import numpy as np
from linearmodels import PanelOLS
import statsmodels.api as sm
from statsmodels.formula.api import ols

# np.set_printoptions(precision=3, suppress=True)

#This will allow all columns to be displayed when reviewing the data.
pd.options.display.max_columns = None

In [2]:
'''
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)
import sys
print(sys.version)
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")
'''

'\nimport tensorflow as tf\n\nfrom tensorflow import keras\nfrom tensorflow.keras import layers\nfrom tensorflow.keras.layers.experimental import preprocessing\n\nprint("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices(\'GPU\')))\ntf.test.is_built_with_cuda()\nprint(tf.version.VERSION)\nimport sys\nprint(sys.version)\ngpu = len(tf.config.list_physical_devices(\'GPU\'))>0\nprint("GPU is", "available" if gpu else "NOT AVAILABLE")\n'

## Load in and manipulate dataset.

Below is for manipulating the dataset before running it through the funciton.

In [3]:
# Load in HMDA Data
HMDA_clean_file_location = r'2019 HMDA Clean IL SAMPLE.csv'
HMDA_clean_0 = pd.read_csv(HMDA_clean_file_location)
HMDA_clean_0

,Year,Lender_LEI,State,County_Code,Census_Tract,Approved,Denied,Race,Ethnicity,Sex,Income,Log_Income,Loan_Amount,Log_Loan_Amount,LTV,Loan_Type,DTI_Ratio,preapproval,Occupancy_Type
0,2019,5Z1UQ1CWY0DQ3KJWDQ07,IL,17197.0,1.719788e+10,1,0,0_White,0_Not Hispanic,Female,185.0,5.220356,255000.0,12.449019,80.000,Conventional,20%-<30%,2,1
1,2019,OTQ7L99FG3H1GQVQBT56,IL,17201.0,1.720100e+10,1,0,0_White,0_Not Hispanic,0_Male,47.0,3.850148,125000.0,11.736069,101.180,Conventional,40,2,1
2,2019,549300U3721PJGQZYY68,IL,17197.0,1.719788e+10,1,0,0_White,0_Not Hispanic,Female,100.0,4.605170,155000.0,11.951180,80.000,Conventional,0%-20%,2,1
3,2019,5493003GQDUH26DNNH17,IL,17163.0,1.716350e+10,1,0,0_White,0_Not Hispanic,0_Male,60.0,4.094345,215000.0,12.278393,100.000,VA,50%-60%,2,1
4,2019,549300UHEEV73TKCZY62,IL,17097.0,1.709786e+10,1,0,0_White,Hispanic,0_Male,57.0,4.043051,145000.0,11.884489,80.000,Conventional,42,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,2019,549300O7SGM8FH65GQ47,IL,17019.0,1.701901e+10,1,0,0_White,0_Not Hispanic,Female,63.0,4.143135,255000.0,12.449019,90.000,Conventional,30%-<36%,2,1
2273,2019,C5654JQHZUHN0772B561,IL,17097.0,1.709786e+10,1,0,0_White,0_Not Hispanic,0_Male,206.0,5.327876,305000.0,12.628067,100.000,VA,46,2,1
2274,2019,549300O7SGM8FH65GQ47,IL,17115.0,1.711500e+10,1,0,0_White,0_Not Hispanic,Female,72.0,4.276666,95000.0,11.461632,89.921,Conventional,30%-<36%,2,1
2275,2019,549300U6DW7DX671T306,IL,17007.0,1.700701e+10,1,0,0_White,Hispanic,Female,202.0,5.308268,115000.0,11.652687,85.000,Conventional,43,2,3


In [4]:
#HMDA_clean.columns

### Check for further cleaning

In [5]:
#HMDA_clean.info()

In [6]:
#Clean df
HMDA_clean_1 = HMDA_clean_0.copy()
HMDA_clean_1 = HMDA_clean_1.dropna()
HMDA_clean_1['Census_Tract'] = HMDA_clean_1['Census_Tract'].apply(str)
#HMDA_clean.info()

Below filters the occupancy type to Principal residence. It omits secondary residence purposes and investment purposes.

In [7]:
# "Occupancy_Type" = 1, Second Residence" = 2, "Investment Property" = 3.
HMDA_clean = HMDA_clean_1[HMDA_clean_1["Occupancy_Type"] == 1]

# Approval on Race/Ethnicity/Sex interactions only.

In [8]:
No_Controls_Model = ols("Approved ~ Race*Ethnicity*Sex", data = HMDA_clean).fit(cov_type = 'Cluster', cov_kwds = {'groups': HMDA_clean['County_Code']})
No_Controls_Model.summary()

C:\Users\raywz\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
C:\Users\raywz\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 10
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Approved   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     89.07
Date:                Fri, 03 Jun 2022   Prob (F-statistic):          8.40e-154
Time:                        14:33:46   Log-Likelihood:                -339.39
No. Observations:                2153   AIC:                             706.8
Df Residuals:                    2139   BIC:                             786.2
Df Model:                          13                                         
Covariance Type:              Cluster                                         
=====================================================================================================================
                                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             0.9310      0.011     86.544      0.000       0.910       0.952
Race[T.Asian]                                        -0.0012      0.027     -0.043      0.965      -0.054       0.052
Race[T.Black]                                        -0.1128      0.019     -5.972      0.000      -0.150      -0.076
Race[T.Other]                                        -0.1310      0.186     -0.704      0.482      -0.496       0.234
Ethnicity[T.Hispanic]                                -0.0515      0.015     -3.430      0.001      -0.081      -0.022
Sex[T.Female]                                        -0.0079      0.013     -0.609      0.543      -0.033       0.018
Race[T.Asian]:Ethnicity[T.Hispanic]                   0.1217      0.030      4.068      0.000       0.063       0.180
Race[T.Black]:Ethnicity[T.Hispanic]                   0.2334      0.026      8.907      0.000       0.182       0.285
Race[T.Other]:Ethnicity[T.Hispanic]                   0.2515      0.188      1.341      0.180      -0.116       0.619
Race[T.Asian]:Sex[T.Female]                           0.0245      0.021      1.145      0.252      -0.017       0.067
Race[T.Black]:Sex[T.Female]                           0.0154      0.023      0.665      0.506      -0.030       0.061
Race[T.Other]:Sex[T.Female]                           0.2079      0.184      1.132      0.257      -0.152       0.568
Ethnicity[T.Hispanic]:Sex[T.Female]                   0.0124      0.041      0.304      0.761      -0.067       0.092
Race[T.Asian]:Ethnicity[T.Hispanic]:Sex[T.Female]          0          0        nan        nan           0           0
Race[T.Black]:Ethnicity[T.Hispanic]:Sex[T.Female]    -0.3532      0.269     -1.313      0.189      -0.881       0.174
Race[T.Other]:Ethnicity[T.Hispanic]:Sex[T.Female]          0          0        nan        nan           0           0
==============================================================================
Omnibus:                     1201.577   Durbin-Watson:                   1.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6163.802
Skew:                          -2.807   Prob(JB):                         0.00
Kurtosis:                       9.099   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# Model 1 - Indicators Only

In [9]:
#omit ['White', 'Not Hispanic', 'Male','DTI_less_than_20']
#don't forget to add census tract, lei, and relationships
Model_1 = ols("Approved ~ Race + Ethnicity + Sex\
          + LTV + DTI_Ratio + Lender_LEI + Census_Tract\
          + Log_Income + Log_Loan_Amount", data = HMDA_clean).fit(cov_type = 'Cluster', cov_kwds = {'groups': HMDA_clean['County_Code']})
Model_1.summary()

C:\Users\raywz\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1623, but rank is 41
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Approved   R-squared:                       0.821
Model:                            OLS   Adj. R-squared:                  0.333
Method:                 Least Squares   F-statistic:                 3.398e+09
Date:                Fri, 03 Jun 2022   Prob (F-statistic):               0.00
Time:                        14:33:55   Log-Likelihood:                 1497.4
No. Observations:                2153   AIC:                             159.1
Df Residuals:                     576   BIC:                             9108.
Df Model:                        1576                                         
Covariance Type:              Cluster                                         
======================================================================================================
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.9813      0.336      2.916      0.004       0.322       1.641
Race[T.Asian]                          0.0224      0.055      0.404      0.686      -0.086       0.131
Race[T.Black]                         -0.0820      0.081     -1.012      0.312      -0.241       0.077
Race[T.Other]                          0.0065      0.056      0.116      0.908      -0.103       0.116
Ethnicity[T.Hispanic]                  0.0038      0.048      0.079      0.937      -0.091       0.099
Sex[T.Female]                         -0.0151      0.047     -0.322      0.747      -0.107       0.077
DTI_Ratio[T.20%-<30%]                 -0.0402      0.073     -0.552      0.581      -0.183       0.103
DTI_Ratio[T.30%-<36%]                  0.0227      0.085      0.266      0.790      -0.145       0.190
DTI_Ratio[T.36]                        0.0321      0.097      0.330      0.741      -0.158       0.223
DTI_Ratio[T.37]                       -0.1007      0.140     -0.719      0.472      -0.375       0.174
DTI_Ratio[T.38]                        0.0280      0.097      0.288      0.773      -0.162       0.218
DTI_Ratio[T.39]                       -0.0046      0.125     -0.037      0.971      -0.249       0.240
DTI_Ratio[T.40]                        0.0012      0.116      0.010      0.992      -0.227       0.229
DTI_Ratio[T.41]                       -0.0921      0.118     -0.782      0.434      -0.323       0.139
DTI_Ratio[T.42]                       -0.0757      0.100     -0.757      0.449      -0.272       0.120
DTI_Ratio[T.43]                       -0.0029      0.107     -0.027      0.979      -0.212       0.206
DTI_Ratio[T.44]                       -0.0542      0.106     -0.513      0.608      -0.261       0.153
DTI_Ratio[T.45]                       -0.0344      0.083     -0.416      0.677      -0.196       0.128
DTI_Ratio[T.46]                        0.1311      0.200      0.655      0.513      -0.261       0.524
DTI_Ratio[T.47]                       -0.0966      0.174     -0.556      0.578      -0.437       0.244
DTI_Ratio[T.48]                       -0.0323      0.075     -0.434      0.664      -0.178       0.114
DTI_Ratio[T.49]                       -0.0384      0.075     -0.511      0.609      -0.185       0.109
DTI_Ratio[T.50%-60%]                  -0.2005      0.161     -1.244      0.214      -0.516       0.115
DTI_Ratio[T.>60%]                     -0.6490      0.173     -3.746      0.000      -0.989      -0.309
Lender_LEI[T.2138007DQF1WY8NDYM18]     0.1110      0.134      0.826      0.409      -0.152       0.374
Lender_LEI[T.213800XR2TCBQJSF1X93]    -0.0778      0.145     -0.536      0.592      -0.362       0.206
Lender_LEI[T.25490018IFQOT83Q7H49]     0.1457      0.337      0.432      0.666      -0.516      

# Model 2 - Race/Ethnicity/Sex Interactions

In [10]:
Model_2 = ols("Approved ~ Race*Ethnicity*Sex\
          + LTV + DTI_Ratio + Lender_LEI + Census_Tract\
          + Log_Income + Log_Loan_Amount", data = HMDA_clean).fit(cov_type = 'Cluster', cov_kwds = {'groups': HMDA_clean['County_Code']})
Model_2.summary()

C:\Users\raywz\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1633, but rank is 41
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Approved   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.327
Method:                 Least Squares   F-statistic:                 2.792e+10
Date:                Fri, 03 Jun 2022   Prob (F-statistic):               0.00
Time:                        14:33:58   Log-Likelihood:                 1501.6
No. Observations:                2153   AIC:                             164.7
Df Residuals:                     569   BIC:                             9153.
Df Model:                        1583                                         
Covariance Type:              Cluster                                         
=====================================================================================================================
                                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             1.0088      0.327      3.089      0.002       0.369       1.649
Race[T.Asian]                                        -0.0035      0.060     -0.059      0.953      -0.121       0.114
Race[T.Black]                                        -0.0665      0.115     -0.578      0.563      -0.292       0.159
Race[T.Other]                                         0.0549      0.121      0.454      0.650      -0.182       0.292
Ethnicity[T.Hispanic]                                -0.0008      0.075     -0.010      0.992      -0.148       0.146
Sex[T.Female]                                        -0.0201      0.052     -0.389      0.697      -0.121       0.081
DTI_Ratio[T.20%-<30%]                                -0.0376      0.070     -0.534      0.594      -0.175       0.100
DTI_Ratio[T.30%-<36%]                                 0.0254      0.081      0.315      0.752      -0.133       0.184
DTI_Ratio[T.36]                                       0.0345      0.095      0.366      0.715      -0.151       0.220
DTI_Ratio[T.37]                                      -0.1038      0.143     -0.727      0.467      -0.384       0.176
DTI_Ratio[T.38]                                       0.0277      0.098      0.283      0.777      -0.164       0.219
DTI_Ratio[T.39]                                   -4.776e-05      0.137     -0.000      1.000      -0.269       0.269
DTI_Ratio[T.40]                                       0.0043      0.116      0.037      0.970      -0.223       0.232
DTI_Ratio[T.41]                                      -0.0848      0.118     -0.719      0.472      -0.316       0.147
DTI_Ratio[T.42]                                      -0.0742      0.100     -0.746      0.456      -0.269       0.121
DTI_Ratio[T.43]                                      -0.0014      0.109     -0.013      0.990      -0.216       0.213
DTI_Ratio[T.44]                                      -0.0555      0.103     -0.540      0.590      -0.257       0.146
DTI_Ratio[T.45]                                      -0.0337      0.083     -0.409      0.683      -0.196       0.128
DTI_Ratio[T.46]                                       0.1300      0.197      0.661      0.509      -0.256       0.516
DTI_Ratio[T.47]                                      -0.0991      0.177     -0.561      0.575      -0.445       0.247
DTI_Ratio[T.48]                                      -0.0275      0.074     -0.374      0.709      -0.172       0.117
DTI_Ratio[T.49]                                      -0.0347      0.074     -0.470      0.638      -0.179       0.110
DTI_Ratio[T.50%-60%]                                 -0.2012      0.160     -1.254      0.210      -0.515       0.113
DTI_Ratio[T.>60

# Model 3 - DTI/LTV Interactions

In [11]:
Model_3 = ols("Approved ~ Race + Ethnicity + Sex\
          + LTV*DTI_Ratio + Lender_LEI + Census_Tract\
          + Log_Income + Log_Loan_Amount", data = HMDA_clean).fit(cov_type = 'Cluster', cov_kwds = {'groups': HMDA_clean['County_Code']})
Model_3.summary()

C:\Users\raywz\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1641, but rank is 41
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Approved   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                -1.957e+11
Date:                Fri, 03 Jun 2022   Prob (F-statistic):               1.00
Time:                        14:34:01   Log-Likelihood:                 1579.3
No. Observations:                2153   AIC:                             31.37
Df Residuals:                     558   BIC:                             9082.
Df Model:                        1594                                         
Covariance Type:              Cluster                                         
======================================================================================================
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.8885      0.436      2.037      0.042       0.033       1.743
Race[T.Asian]                          0.0043      0.059      0.072      0.943      -0.111       0.120
Race[T.Black]                         -0.0782      0.076     -1.034      0.301      -0.227       0.070
Race[T.Other]                         -0.0073      0.041     -0.180      0.857      -0.087       0.072
Ethnicity[T.Hispanic]                  0.0003      0.051      0.006      0.996      -0.099       0.100
Sex[T.Female]                         -0.0137      0.041     -0.331      0.740      -0.095       0.067
DTI_Ratio[T.20%-<30%]                  0.1007      0.338      0.298      0.766      -0.562       0.763
DTI_Ratio[T.30%-<36%]                  0.1393      0.279      0.498      0.618      -0.409       0.687
DTI_Ratio[T.36]                       -0.0429      0.470     -0.091      0.927      -0.965       0.879
DTI_Ratio[T.37]                       -0.2529      0.796     -0.318      0.751      -1.812       1.307
DTI_Ratio[T.38]                        0.3195      0.561      0.569      0.569      -0.780       1.419
DTI_Ratio[T.39]                        0.4185      0.566      0.739      0.460      -0.692       1.529
DTI_Ratio[T.40]                        0.3302      0.464      0.711      0.477      -0.580       1.240
DTI_Ratio[T.41]                        0.6115      1.025      0.597      0.551      -1.397       2.620
DTI_Ratio[T.42]                       -0.1237      0.700     -0.177      0.860      -1.495       1.248
DTI_Ratio[T.43]                        0.2575      0.441      0.584      0.559      -0.607       1.122
DTI_Ratio[T.44]                        0.5423      0.562      0.965      0.334      -0.559       1.643
DTI_Ratio[T.45]                        0.3488      0.429      0.813      0.416      -0.492       1.190
DTI_Ratio[T.46]                       -0.1170      0.273     -0.429      0.668      -0.652       0.418
DTI_Ratio[T.47]                        0.3792      0.488      0.778      0.437      -0.576       1.335
DTI_Ratio[T.48]                        0.3837      0.453      0.848      0.397      -0.503       1.271
DTI_Ratio[T.49]                        0.3037      0.343      0.885      0.376      -0.369       0.976
DTI_Ratio[T.50%-60%]                  -2.0819      0.858     -2.428      0.015      -3.763      -0.401
DTI_Ratio[T.>60%]                      0.3598      1.998      0.180      0.857      -3.556       4.276
Lender_LEI[T.2138007DQF1WY8NDYM18]    -0.1007      0.235     -0.429      0.668      -0.561       0.359
Lender_LEI[T.213800XR2TCBQJSF1X93]    -0.0439      0.148     -0.296      0.767      -0.334       0.246
Lender_LEI[T.25490018IFQOT83Q7H49]     0.1807      0.346      0.522      0.602      -0.498      

# Model 4 - Lender and Census_Tract Interaction

In [ ]:
Model_4 = ols("Approved ~ Race + Ethnicity + Sex\
          + LTV + DTI_Ratio + Lender_LEI*Census_Tract\
          + Log_Income + Log_Loan_Amount", data = HMDA_clean).fit(cov_type = 'Cluster', cov_kwds = {'groups': HMDA_clean['County_Code']})
Model_4.summary()

# Model 5 - All Interactions(minus LEI/census_tract) and Control Variables.

In [12]:
Model_5 = ols("Approved ~ Race*Ethnicity*Sex\
          + LTV*DTI_Ratio + Lender_LEI + Census_Tract\
          + Log_Income + Log_Loan_Amount\
          + C(preapproval) + C(Loan_Type)", data = HMDA_clean).fit(cov_type = 'Cluster', cov_kwds = {'groups': HMDA_clean['County_Code']})
Model_5.summary()

C:\Users\raywz\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1655, but rank is 41
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Approved   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                -4.387e+11
Date:                Fri, 03 Jun 2022   Prob (F-statistic):               1.00
Time:                        14:34:03   Log-Likelihood:                 1608.5
No. Observations:                2153   AIC:                            -4.956
Df Residuals:                     547   BIC:                             9108.
Df Model:                        1605                                         
Covariance Type:              Cluster                                         
=====================================================================================================================
                                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             1.1296      0.380      2.976      0.003       0.386       1.874
Race[T.Asian]                                        -0.0373      0.066     -0.561      0.575      -0.168       0.093
Race[T.Black]                                        -0.0546      0.092     -0.594      0.553      -0.235       0.126
Race[T.Other]                                        -0.0132      0.107     -0.123      0.902      -0.223       0.197
Ethnicity[T.Hispanic]                                -0.0003      0.074     -0.004      0.997      -0.145       0.145
Sex[T.Female]                                        -0.0132      0.043     -0.304      0.761      -0.098       0.072
DTI_Ratio[T.20%-<30%]                                 0.1172      0.330      0.355      0.722      -0.529       0.764
DTI_Ratio[T.30%-<36%]                                 0.1412      0.292      0.483      0.629      -0.432       0.714
DTI_Ratio[T.36]                                      -0.0082      0.490     -0.017      0.987      -0.968       0.951
DTI_Ratio[T.37]                                      -0.2958      0.792     -0.374      0.709      -1.847       1.256
DTI_Ratio[T.38]                                       0.3312      0.561      0.591      0.555      -0.768       1.430
DTI_Ratio[T.39]                                       0.3992      0.654      0.611      0.541      -0.882       1.680
DTI_Ratio[T.40]                                       0.3160      0.492      0.642      0.521      -0.649       1.281
DTI_Ratio[T.41]                                       0.6239      1.070      0.583      0.560      -1.474       2.722
DTI_Ratio[T.42]                                      -0.0707      0.581     -0.122      0.903      -1.210       1.068
DTI_Ratio[T.43]                                       0.2306      0.505      0.457      0.648      -0.759       1.220
DTI_Ratio[T.44]                                       0.5649      0.681      0.830      0.407      -0.770       1.899
DTI_Ratio[T.45]                                       0.4205      0.470      0.895      0.371      -0.501       1.342
DTI_Ratio[T.46]                                      -0.1548      0.256     -0.605      0.545      -0.657       0.347
DTI_Ratio[T.47]                                       0.2643      0.528      0.501      0.617      -0.770       1.299
DTI_Ratio[T.48]                                       0.3801      0.459      0.828      0.408      -0.520       1.280
DTI_Ratio[T.49]                                       0.3266      0.406      0.804      0.422      -0.470       1.123
DTI_Ratio[T.50%-60%]                                 -2.0873      0.794     -2.630      0.009      -3.643      -0.532
DTI_Ratio[T.>60

# Denied Model 1

In [13]:
Denied_1 = ols("Denied ~ Race + Ethnicity + Sex\
          + LTV + DTI_Ratio + Lender_LEI + Census_Tract\
          + Log_Income + Log_Loan_Amount", data = HMDA_clean).fit(cov_type = 'Cluster', cov_kwds = {'groups': HMDA_clean['County_Code']})
Denied_1.summary()

C:\Users\raywz\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1623, but rank is 41
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Denied   R-squared:                       0.821
Model:                            OLS   Adj. R-squared:                  0.333
Method:                 Least Squares   F-statistic:                -1.885e+10
Date:                Fri, 03 Jun 2022   Prob (F-statistic):               1.00
Time:                        14:34:06   Log-Likelihood:                 1497.4
No. Observations:                2153   AIC:                             159.1
Df Residuals:                     576   BIC:                             9108.
Df Model:                        1576                                         
Covariance Type:              Cluster                                         
======================================================================================================
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.0187      0.336      0.056      0.956      -0.641       0.678
Race[T.Asian]                         -0.0224      0.055     -0.404      0.686      -0.131       0.086
Race[T.Black]                          0.0820      0.081      1.012      0.312      -0.077       0.241
Race[T.Other]                         -0.0065      0.056     -0.116      0.908      -0.116       0.103
Ethnicity[T.Hispanic]                 -0.0038      0.048     -0.079      0.937      -0.099       0.091
Sex[T.Female]                          0.0151      0.047      0.322      0.747      -0.077       0.107
DTI_Ratio[T.20%-<30%]                  0.0402      0.073      0.552      0.581      -0.103       0.183
DTI_Ratio[T.30%-<36%]                 -0.0227      0.085     -0.266      0.790      -0.190       0.145
DTI_Ratio[T.36]                       -0.0321      0.097     -0.330      0.741      -0.223       0.158
DTI_Ratio[T.37]                        0.1007      0.140      0.719      0.472      -0.174       0.375
DTI_Ratio[T.38]                       -0.0280      0.097     -0.288      0.773      -0.218       0.162
DTI_Ratio[T.39]                        0.0046      0.125      0.037      0.971      -0.240       0.249
DTI_Ratio[T.40]                       -0.0012      0.116     -0.010      0.992      -0.229       0.227
DTI_Ratio[T.41]                        0.0921      0.118      0.782      0.434      -0.139       0.323
DTI_Ratio[T.42]                        0.0757      0.100      0.757      0.449      -0.120       0.272
DTI_Ratio[T.43]                        0.0029      0.107      0.027      0.979      -0.206       0.212
DTI_Ratio[T.44]                        0.0542      0.106      0.513      0.608      -0.153       0.261
DTI_Ratio[T.45]                        0.0344      0.083      0.416      0.677      -0.128       0.196
DTI_Ratio[T.46]                       -0.1311      0.200     -0.655      0.513      -0.524       0.261
DTI_Ratio[T.47]                        0.0966      0.174      0.556      0.578      -0.244       0.437
DTI_Ratio[T.48]                        0.0323      0.075      0.434      0.664      -0.114       0.178
DTI_Ratio[T.49]                        0.0384      0.075      0.511      0.609      -0.109       0.185
DTI_Ratio[T.50%-60%]                   0.2005      0.161      1.244      0.214      -0.115       0.516
DTI_Ratio[T.>60%]                      0.6490      0.173      3.746      0.000       0.309       0.989
Lender_LEI[T.2138007DQF1WY8NDYM18]    -0.1110      0.134     -0.826      0.409      -0.374       0.152
Lender_LEI[T.213800XR2TCBQJSF1X93]     0.0778      0.145      0.536      0.592      -0.206       0.362
Lender_LEI[T.25490018IFQOT83Q7H49]    -0.1457      0.337     -0.432      0.666      -0.807      

# Denied Model 2 - Add other interactions and control variables.

In [14]:
Denied_2 = ols("Denied ~ Race*Ethnicity*Sex\
          + LTV*DTI_Ratio + Lender_LEI + Census_Tract\
          + Log_Income + Log_Loan_Amount\
          + C(preapproval) + C(Loan_Type)", data = HMDA_clean).fit(cov_type = 'Cluster', cov_kwds = {'groups': HMDA_clean['County_Code']})
Denied_2.summary()

C:\Users\raywz\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1655, but rank is 41
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Denied   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                 7.624e+10
Date:                Fri, 03 Jun 2022   Prob (F-statistic):               0.00
Time:                        14:34:09   Log-Likelihood:                 1608.5
No. Observations:                2153   AIC:                            -4.956
Df Residuals:                     547   BIC:                             9108.
Df Model:                        1605                                         
Covariance Type:              Cluster                                         
=====================================================================================================================
                                                        coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                            -0.1296      0.380     -0.341      0.733      -0.874       0.614
Race[T.Asian]                                         0.0373      0.066      0.561      0.575      -0.093       0.168
Race[T.Black]                                         0.0546      0.092      0.594      0.553      -0.126       0.235
Race[T.Other]                                         0.0132      0.107      0.123      0.902      -0.197       0.223
Ethnicity[T.Hispanic]                                 0.0003      0.074      0.004      0.997      -0.145       0.145
Sex[T.Female]                                         0.0132      0.043      0.304      0.761      -0.072       0.098
DTI_Ratio[T.20%-<30%]                                -0.1172      0.330     -0.355      0.722      -0.764       0.529
DTI_Ratio[T.30%-<36%]                                -0.1412      0.292     -0.483      0.629      -0.714       0.432
DTI_Ratio[T.36]                                       0.0082      0.490      0.017      0.987      -0.951       0.968
DTI_Ratio[T.37]                                       0.2958      0.792      0.374      0.709      -1.256       1.847
DTI_Ratio[T.38]                                      -0.3312      0.561     -0.591      0.555      -1.430       0.768
DTI_Ratio[T.39]                                      -0.3992      0.654     -0.611      0.541      -1.680       0.882
DTI_Ratio[T.40]                                      -0.3160      0.492     -0.642      0.521      -1.281       0.649
DTI_Ratio[T.41]                                      -0.6239      1.070     -0.583      0.560      -2.722       1.474
DTI_Ratio[T.42]                                       0.0707      0.581      0.122      0.903      -1.068       1.210
DTI_Ratio[T.43]                                      -0.2306      0.505     -0.457      0.648      -1.220       0.759
DTI_Ratio[T.44]                                      -0.5649      0.681     -0.830      0.407      -1.899       0.770
DTI_Ratio[T.45]                                      -0.4205      0.470     -0.895      0.371      -1.342       0.501
DTI_Ratio[T.46]                                       0.1548      0.256      0.605      0.545      -0.347       0.657
DTI_Ratio[T.47]                                      -0.2643      0.528     -0.501      0.617      -1.299       0.770
DTI_Ratio[T.48]                                      -0.3801      0.459     -0.828      0.408      -1.280       0.520
DTI_Ratio[T.49]                                      -0.3266      0.406     -0.804      0.422      -1.123       0.470
DTI_Ratio[T.50%-60%]                                  2.0873      0.794      2.630      0.009       0.532       3.643
DTI_Ratio[T.>60